# Setup Logging & Imports

In [ ]:
DEBUG = True

### Setting up logging

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("pipeline")

# Show info messages if DEBUG mode is enabled
if DEBUG:
    logger.setLevel(logging.DEBUG)
    logger.debug("DEBUG mode is enabled. Detailed logs will be shown.")
else:
    logger.setLevel(logging.INFO)
    logger.info("DEBUG mode is disabled. Only essential logs will be shown.")

### Imports

In [ ]:
import os
import torch

# We set up CUDA first to ensure it is configured correctly
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
if torch.cuda.is_available():
    CUDA_DEVICE = torch.device("cuda:0")
    logger.info(f"CUDA is available. Using device: {CUDA_DEVICE}")
else:
    logger.error("CUDA is not available. Please check your PyTorch installation. Using CPU instead...this will be slow.")
    CUDA_DEVICE = torch.device("cpu")

In [ ]:
from pipeline.proj import load_projection_mat, reformat_sinogram, interpolate_projections, pad_and_reshape, divide_sinogram
from pipeline.aggregate_prj import aggregate_saved_projections
from pipeline.aggregate_ct import aggregate_saved_recons
from pipeline.apply_model import apply_model_to_projections, load_model
# from .infer3d import inference_3d
from pipeline.utils import ensure_dir, read_scans_agg_file
from pipeline.paths import Directories, Files
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import yaml
import importlib
import copy
from tqdm import tqdm
import gc
import tigre.utilities.gpu as gpu
from pipeline.FDK_half.FDK_half import FDKHalf
from pipeline.utils import get_geometry

# TODO run FDK via: FFrecon_reconFDK(input_mat, output_mat); in file "FFrecon_fullFDK.m"

# Configuration

In [ ]:
# Scans to convert to PyTorch tensors
# Put None if you don't have any scans to convert
# See the README for how to write this file correctly
# NOTE: This will throw an error if the scan has already been converted
#       If you would like to re-convert a scan,
#       you can delete the file manually
# SCANS_CONVERT = 'scans_convert_to_pt.txt'
SCANS_CONVERT = None

# Phase of the project (all data, models, etc. will be saved under this phase)
PHASE = "7"

# If this data version already exists in this phase, it will be loaded
# Otherwise it will be created using whatever the most updated data creation script is
DATA_VERSION = '13'

# Scans to use for training, val, and testing
# You should set this even if you are not doing aggregation
# See the README for how to write this file correctly
# NOTE: This will NOT throw an error if there are already aggregated scans
#       it will just give a warning and skip the aggregation step
SCANS_AGG = 'scans_to_agg.txt'
# SCANS_AGG = None

# Whether to augment the data for the image domain
# This will only be used if you are doing image domain aggregation
AUGMENT_ID = True

# List of yaml files that contain configurations for the pipeline
# Each file should contain the paramters for a specific model/ensemble
CONFIG_FILES = [
    # "config_01epoch.yaml",
    "config_20epoch.yaml",
]

# Base directory
WORK_ROOT = "D:/NoahSilverberg/ngCBCT"

# NSG_CBCT Path where the raw matlab data is stored
NSG_CBCT_PATH = "D:/MitchellYu/NSG_CBCT"

# Directory with all files specific to this phase/data version
PHASE_DATAVER_DIR = os.path.join(
    WORK_ROOT, f"phase{PHASE}", f"DS{DATA_VERSION}"
)

DIRECTORIES = Directories(
    mat_projections_dir=os.path.join(NSG_CBCT_PATH, "data/prj/HF/mat"),
    pt_projections_dir=os.path.join(WORK_ROOT, "prj_pt"),
    projections_aggregate_dir=os.path.join(PHASE_DATAVER_DIR, "aggregates", "projections"),
    projections_model_dir=os.path.join(PHASE_DATAVER_DIR, "models", "projections"),
    projections_results_dir=os.path.join(PHASE_DATAVER_DIR, "results", "projections"),
    projections_gated_dir=os.path.join(WORK_ROOT, "gated", "prj_mat"),
    reconstructions_dir=os.path.join(PHASE_DATAVER_DIR, "reconstructions"),
    reconstructions_gated_dir=os.path.join(WORK_ROOT, "gated", "fdk_recon"),
    images_aggregate_dir=os.path.join(PHASE_DATAVER_DIR, "aggregates", "images"),
    images_model_dir=os.path.join(PHASE_DATAVER_DIR, "models", "images"),
    images_results_dir=os.path.join(PHASE_DATAVER_DIR, "results", "images"),
)

FILES = Files(DIRECTORIES)

# Data Preparation: projection interpolation

In [ ]:
if SCANS_CONVERT is not None:
    # Read the scans to convert file
    with open(SCANS_CONVERT, "r") as f:
        SCANS_CONVERT = []
        for line in f:
            line = line.strip()
            if not line:
                continue
            patient, scan, scan_type = line.split()
            SCANS_CONVERT.append((patient, scan, scan_type))

    logger.debug(f"Loaded scan list for conversion: {SCANS_CONVERT}")

    logger.info("Starting to process projection data...")

    for patient, scan, scan_type in SCANS_CONVERT:
        g_path = FILES.get_projection_pt_filepath(patient, scan, scan_type, gated=True)
        ng_path = FILES.get_projection_pt_filepath(patient, scan, scan_type, gated=False)

        # Make sure the files do not already exist
        if os.path.exists(g_path) or os.path.exists(ng_path):
            logger.warning(f"Projection files already exist for patient {patient}, scan {scan}, type {scan_type}. Skipping...")
            continue

        # Load the projection data from the matlab files
        mat_path = FILES.get_projection_mat_filepath(patient, scan, scan_type)
        odd_index, angles, prj = load_projection_mat(mat_path)

        # Log shapes of loaded data
        logger.debug(f'Processing patient {patient}, scan {scan}, type {scan_type}')
        logger.debug(f'Loaded odd_index shape: {odd_index.shape}')
        logger.debug(f'Loaded angles shape: {angles.shape}')
        logger.debug(f'Loaded projection shape: {prj.shape}')

        # Flip and permute to get it in the right format
        prj_gcbct, angles1 = reformat_sinogram(prj, angles)

        # Log shapes after reformatting
        logger.debug(f'Reformatted projection shape: {prj_gcbct.shape}')

        # Simulate ngCBCT projections
        prj_ngcbct_li = interpolate_projections(prj_gcbct, odd_index)

        # Log shapes after interpolation
        logger.debug(f'Interpolated ngCBCT projection shape: {prj_ngcbct_li.shape}')

        # Split the projections into two halves so they are good dimensions for the CNN
        combined_gcbct = divide_sinogram(pad_and_reshape(prj_gcbct), v_dim=512 if scan_type == "HF" else 256)
        combined_ngcbct = divide_sinogram(pad_and_reshape(prj_ngcbct_li), v_dim=512 if scan_type == "HF" else 256)

        # Log shapes after dividing sinograms
        logger.debug(f'Combined gCBCT shape: {combined_gcbct.shape}')
        logger.debug(f'Combined ngCBCT shape: {combined_ngcbct.shape}')

        logger.debug(f'Saving projections...')
        
        # Save the projections
        torch.save(combined_gcbct, g_path)
        torch.save(combined_ngcbct, ng_path)

        logger.debug(f'Done with patient {patient}, scan {scan}, type {scan_type}\n')

    logger.info("All projections saved successfully.")

    # Free up memory
    del odd_index, angles, prj, prj_gcbct, angles1, prj_ngcbct_li, combined_gcbct, combined_ngcbct
else:
    logger.info("No scans to convert. Skipping projection data processing.")

### DEBUG: Sample projections

In [ ]:
if DEBUG and SCANS_CONVERT is not None:
    # Pick the first HF scan and first FF scan
    hf_scan = None
    ff_scan = None
    for patient, scan, scan_type in SCANS_CONVERT:
        if scan_type == "HF":
            hf_scan = (patient, scan, scan_type)
            break
    for patient, scan, scan_type in SCANS_CONVERT:
        if scan_type == "FF":
            ff_scan = (patient, scan, scan_type)
            break

    # Display the first HF scan
    # Show the gated and nonstop-gated on subplots
    if hf_scan:
        hf_patient, hf_scan_num, hf_scan_type = hf_scan
        g_path = FILES.get_projection_pt_filepath(hf_patient, hf_scan_num, hf_scan_type, gated=True)
        ng_path = FILES.get_projection_pt_filepath(hf_patient, hf_scan_num, hf_scan_type, gated=False)
        hf_gated_prj = torch.load(g_path)
        hf_ng_prj = torch.load(ng_path)
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(hf_gated_prj[0, 0, :, :].cpu().numpy(), cmap='gray')
        plt.title(f'Gated Projection - {hf_scan_type} p{hf_patient}_{hf_scan_num}')
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.imshow(hf_ng_prj[0, 0, :, :].cpu().numpy(), cmap='gray')
        plt.title(f'Nonstop-Gated Projection - {hf_scan_type} p{hf_patient}_{hf_scan_num}')
        plt.axis('off')
        plt.tight_layout()
        plt.show()

        # Free up memory
        del hf_gated_prj, hf_ng_prj

    # Repeat for FF scan
    if ff_scan:
        ff_patient, ff_scan_num, ff_scan_type = ff_scan
        g_path = FILES.get_projection_pt_filepath(ff_patient, ff_scan_num, ff_scan_type, gated=True)
        ff_gated_prj = torch.load(g_path)
        ng_path = FILES.get_projection_pt_filepath(ff_patient, ff_scan_num, ff_scan_type, gated=False)
        ff_ng_prj = torch.load(ng_path)
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(ff_gated_prj[0, 0, :, :].cpu().numpy(), cmap='gray')
        plt.title(f'Gated Projection - {ff_scan_type} p{ff_patient}_{ff_scan_num}')
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.imshow(ff_ng_prj[0, 0, :, :].cpu().numpy(), cmap='gray')
        plt.title(f'Nonstop-Gated Projection - {ff_scan_type} p{ff_patient}_{ff_scan_num}')
        plt.axis('off')
        plt.tight_layout()
        plt.show()

        # Free up memory
        del ff_gated_prj, ff_ng_prj

# Aggregate projections for train/val/test

In [ ]:
if SCANS_AGG is not None:
    scans_agg, scan_type = read_scans_agg_file(SCANS_AGG)
    logger.debug(f"Loaded scan list for aggregation: {scans_agg}")

    # Only aggregate projections if they don't already exist
    agg_dir = DIRECTORIES.projections_aggregate_dir
    if agg_dir is None:
        logger.warning("No aggregation directory specified. Skipping projection data aggregation.")
    elif len(os.listdir(agg_dir)) > 0:
        logger.warning(f"Aggregated projection data already exists in {agg_dir}. Skipping...")
    else:
        logger.info("Starting to aggregate projection data...")
        
        # Aggregate and save projection data sets
        for split in ['TRAIN', 'VALIDATION', 'TEST']:
            if len(scans_agg[split]) > 0:
                ng_paths = [FILES.get_projection_pt_filepath(patient, scan, scan_type, gated=False) for patient, scan, scan_type in scans_agg[split]]
                prj_ngcbct = aggregate_saved_projections(ng_paths)
                ng_agg_path = FILES.get_projections_aggregate_filepath(split, gated=False)
                np.save(ng_agg_path, prj_ngcbct.numpy())
                del prj_ngcbct
                logger.debug("Done with nonstop-gated...")
                g_paths = [FILES.get_projection_pt_filepath(patient, scan, scan_type, gated=True) for patient, scan, scan_type in scans_agg[split]]
                prj_gcbct = aggregate_saved_projections(g_paths)
                g_agg_path = FILES.get_projections_aggregate_filepath(split, gated=True)
                np.save(g_agg_path, prj_gcbct.numpy())
                del prj_gcbct
                logger.debug("Done with gated...")

                logger.debug(f"Aggregated projections saved for {scan_type} {split}.\n")
            else:
                logger.debug(f"No scans to aggregate for {scan_type} {split}. Skipping aggregation.")

        # Free up memory
        del prj_gcbct, prj_ngcbct

        logger.info("Projection data aggregation completed successfully.")
        logger.info("Aggregated projection data saved in: %s", agg_dir)
else:
    logger.info("No scans to aggregate. Skipping projection data aggregation.")

# Training PD CNN

In [ ]:
for config_file in CONFIG_FILES:
    # Load the yaml configuration file
    with open(config_file, "r") as f:
        config = yaml.safe_load(f)
    
    logger.debug(f"Loaded configuration from {config_file}")

    # Skip this config if the user has set PD_training to False
    if not config['PD_settings']['training']:
        logger.info(f"Skipping PD training for {config_file} as PD training is set to False.")
        continue

    # Get the training application
    module_name, class_name = config['PD_settings']['training_app'].rsplit('.', 1)
    module = importlib.import_module("pipeline." + module_name)
    cls = getattr(module, class_name)

    logger.debug(f"Loaded class {class_name} from module {module_name}")

    # Get the model version (for naming purposes)
    model_version = config['PD_settings']['model_version']

    # Get the ensemble size, and loop through it
    ensemble_size = config['PD_settings']['ensemble_size']
    for i in range(ensemble_size):
        # If we are training an ensemble, we add an identifier to the model version
        if ensemble_size > 1:
            # Deepcopy config so we don't affect the original
            cfg = copy.deepcopy(config)
            cfg['PD_settings']['model_version'] = f"{model_version}_{i+1:02}" # e.g., "v1_01"
        else:
            cfg = config

        # Add the data version to the configuration
        cfg['PD_settings']['data_version'] = DATA_VERSION

        # Instantiate with the loaded configuration
        instance = cls(cfg, "PROJ", DEBUG, FILES)

        logger.info(f"Going to try training the {i + 1}-th model with configuration from {config_file}...")

        # Run the training
        instance.main()

        logger.info(f"Finished training the {i + 1}-th model.\n")

        del instance, cfg
        gc.collect()

    # Free up memory
    del module, cls, config, module_name, class_name

# Apply PD model to all nonstop-gated sinograms

In [ ]:
if SCANS_AGG is None:
    logger.info("Skipping model application as the aggregation scan list is not provided.")
else:
    # Loop through the configurations again
    for config_file in CONFIG_FILES:
        # Load the yaml configuration file
        with open(config_file, "r") as f:
            config = yaml.safe_load(f)

        logger.debug(f"Loaded configuration from {config_file}")

        # Get the ensemble size, and loop through it
        ensemble_size = config['PD_settings']['ensemble_size']
        for i in range(ensemble_size):
            model_version = config['PD_settings']['model_version']

            # If we are training an ensemble, we add an identifier to the model version
            if ensemble_size > 1:
                model_version = f"{model_version}_{i+1:02}"

            scan_type_model = config['PD_settings']['scan_type']

            # Load the trained PD model onto the GPU
            model_path = FILES.get_model_filepath(model_version, "PROJ")
            PD_model = load_model(config['PD_settings']['network_name'], model_path, CUDA_DEVICE)

            scans_agg, scan_type = read_scans_agg_file(SCANS_AGG, list_=True)
            if scan_type != config['PD_settings']['scan_type']:
                raise ValueError(f"Scan type in aggregation file ({scan_type}) does not match scan type in config ({config['PD_settings']['scan_type']}).")
            
            for patient, scan, scan_type in tqdm(scans_agg, desc=f"Applying model {model_version} to projections"):
                # Get the matlab dicts for the ground truth and CNN projections
                mat_path = FILES.get_projection_mat_filepath(patient, scan, scan_type)
                gated_pt_path = FILES.get_projection_pt_filepath(patient, scan, scan_type, gated=True)
                ng_pt_path = FILES.get_projection_pt_filepath(patient, scan, scan_type, gated=False)
                
                # Save the ground truth and CNN projections
                gated_path = FILES.get_projections_results_filepath(model_version, patient, scan, scan_type, gated=True)
                ng_path = FILES.get_projections_results_filepath(model_version, patient, scan, scan_type, gated=False)

                if os.path.exists(ng_path):
                    logger.info(f"Projections results already exist for {scan_type} p{patient}_{scan} for model {model_version}. Skipping...")
                    continue

                g_mat, cnn_mat = apply_model_to_projections(PD_model, scan_type, mat_path, gated_pt_path, ng_pt_path, CUDA_DEVICE)

                if os.path.exists(gated_path):
                    logger.info(f"Ground-truth projections already exist for {scan_type} p{patient}_{scan}. Skipping...")
                else:
                    scipy.io.savemat(gated_path, g_mat)
                scipy.io.savemat(ng_path, cnn_mat)

                logger.debug(f"Saved projections for {scan_type} p{patient}_{scan}.")

                del g_mat, cnn_mat

            # Free up memory
            del PD_model

    logger.info("All models applied to projections.")

# Apply FDK to all projections that were processed by the PD model

In [ ]:
# TODO do we do this differently for FF?
gpuids = gpu.getGpuIds()
gpuids.devices = [0]
geo = get_geometry()

# Loop through the configurations again
for config_file in CONFIG_FILES:
    # Load the yaml configuration file
    with open(config_file, "r") as f:
        config = yaml.safe_load(f)

    logger.debug(f"Loaded configuration from {config_file}")

    # Get the ensemble size, and loop through it
    ensemble_size = config['PD_settings']['ensemble_size']
    for i in range(ensemble_size):
        model_version = config['PD_settings']['model_version']

        # If we are training an ensemble, we add an identifier to the model version
        if ensemble_size > 1:
            model_version = f"{model_version}_{i+1:02}"

        scan_type_model = config['PD_settings']['scan_type']

        # Load the trained PD model onto the GPU
        model_path = FILES.get_model_filepath(model_version, "PROJ")
        PD_model = load_model(config['PD_settings']['network_name'], model_path, torch.device(CUDA_DEVICE))
            
        scans_agg, scan_type = read_scans_agg_file(SCANS_AGG, list_=True)
        if scan_type != config['PD_settings']['scan_type']:
            raise ValueError(f"Scan type in aggregation file ({scan_type}) does not match scan type in config ({config['PD_settings']['scan_type']}).")

        for patient, scan, scan_type in tqdm(scans_agg, desc=f"Applying FDK to gated and nonstop-gated projections for model {model_version}"):
            for gated in [True, False]:
                # Load the scan and perform FDK reconstruction
                result_path = FILES.get_projections_results_filepath(model_version, patient, scan, scan_type, gated=gated)
                
                recon_path = FILES.get_recon_filepath(model_version, patient, scan, scan_type, gated=gated)
                if os.path.exists(recon_path):
                    logger.info(f"{'Gated' if gated else 'Nonstop-gated'} reconstruction already exists for {scan_type} p{patient}_{scan}. Skipping...")
                    continue

                mat = scipy.io.loadmat(result_path)
                fdk_half = FDKHalf()(mat['prj'], get_geometry(), mat['angles'].flatten(), filter='hann', parker=True)
                fdk_half = torch.from_numpy(fdk_half).detach()

                # Save the recon results as .pt
                torch.save(fdk_half, recon_path)

                logger.debug(f"Saved {'gated' if gated else 'nonstop-gated'} projection for {scan_type} p{patient}_{scan}.")

                # Delete the nonstop-gated result to free storage space
                if not gated:
                    os.remove(result_path)
                    logger.debug(f"Deleted nonstop-gated projection file {result_path} to save storage space.")

                del fdk_half, mat

        # Free up memory
        del PD_model

logger.info("All models applied to projections.")

# Aggregate CT volumes for train/val/test

In [ ]:
if SCANS_AGG is not None:
    scans_agg, scan_type = read_scans_agg_file(SCANS_AGG)
    logger.debug(f"Loaded scan list for aggregation: {scans_agg}")

    # Only aggregate reconstructions if they don't already exist
    if len([f for f in os.listdir(AGG_DIR) if f.startswith("FDK")]) > 0:
        raise FileExistsError(f"Aggregated reconstruction data for phase {PHASE} data version {DATA_VERSION} already exists in {AGG_DIR}. Please delete the existing files or change the data version to re-aggregate.")
    else:
        logger.info("Starting to aggregate reconstruction data...")

        pt_recon_dir = os.path.join(WORK_ROOT, 'data_pt', 'recon')

        # Aggregate and save reconstruction data sets
        for sample in ['TRAIN', 'VALIDATION', 'TEST']:
            if len(scans_agg[sample]):
                recon_ngcbct = aggregate_saved_recons(scan_type, sample, pt_recon_dir, scans_agg, truth=False, augment=AUGMENT_ID)
                np.save(os.path.join(AGG_DIR, f"FDK_IMAG_ng_{scan_type}_{sample}.npy"), recon_ngcbct.numpy()) # e.g., IMAG_ng_HF_TRAIN.npy
                del recon_ngcbct
                logger.debug("Done with nonstop-gated...")
                recon_gcbct = aggregate_saved_recons(scan_type, sample, pt_recon_dir, scans_agg, truth=True, augment=AUGMENT_ID)
                np.save(os.path.join(AGG_DIR, f"FDK_IMAG_gated_{scan_type}_{sample}.npy"), recon_gcbct.numpy())
                del recon_gcbct
                logger.debug("Done with gated...")

                logger.debug(f"Aggregated reconstructions saved for {scan_type} {sample}.\n")
            else:
                logger.debug(f"No scans to aggregate for {scan_type} {sample}. Skipping aggregation.")

        # Free up memory
        del recon_gcbct, recon_ngcbct

    logger.info("Reconstruction data aggregation completed successfully.")
    logger.info("Aggregated reconstruction data saved in: %s", AGG_DIR)
else:
    logger.info("No scans to aggregate. Skipping reconstruction data aggregation.")

# Train ID CNN

In [ ]:
for config_file in CONFIG_FILES:
    # Load the yaml configuration file
    with open(config_file, "r") as f:
        config = yaml.safe_load(f)
    
    logger.debug(f"Loaded configuration from {config_file}")

    # Skip this config if the user has set ID_training to False
    if not config['ID_settings']['training']:
        logger.info(f"Skipping ID training for {config_file} as ID training is set to False.")
        continue

    # Get the training application
    module_name, class_name = config['ID_settings']['training_app'].rsplit('.', 1)
    module = importlib.import_module("pipeline." + module_name)
    cls = getattr(module, class_name)

    logger.debug(f"Loaded class {class_name} from module {module_name}")

    # Get the model version (for naming purposes)
    model_version = config['ID_settings']['model_version']

    # Get the ensemble size, and loop through it
    ensemble_size = config['ID_settings']['ensemble_size']
    for i in range(ensemble_size):
        # If we are training an ensemble, we add an identifier to the model version
        if ensemble_size > 1:
            # Deepcopy config so we don't affect the original
            cfg = copy.deepcopy(config)
            cfg['ID_settings']['model_version'] = f"{model_version}_{i+1:02}" # e.g., "v1_01"
        else:
            cfg = config

        # Add the data version to the configuration
        cfg['ID_settings']['data_version'] = DATA_VERSION

        # Instantiate with the loaded configuration
        instance = cls(cfg, "IMAG", DEBUG, MODEL_DIR, AGG_DIR)

        logger.info(f"Going to try training the {i + 1}-th model with configuration from {config_file}...")

        # Run the training
        instance.main()

        logger.info(f"Finished training the {i + 1}-th model.\n")

        del instance, cfg
        gc.collect()

    # Free up memory
    del module, cls, config, module_name, class_name

# Pass all samples through the ID model